In [ ]:
from google.colab import drive
import os
# Install p7zip-full for handling .7z archives
!apt-get install -y p7zip-full
!pip install nibabel nilearn dicom2nifti
from scipy.ndimage import shift

from nilearn.image import clean_img, resample_to_img
import nibabel as nib
import os
import dicom2nifti
import nibabel as nib
from nilearn.image import clean_img, resample_to_img, smooth_img, resample_img
from nilearn.masking import compute_brain_mask
from nilearn.datasets import load_mni152_template
import numpy as np
from nilearn.decomposition import CanICA
from sklearn.preprocessing import StandardScaler
from nilearn.connectome import ConnectivityMeasure
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import pandas as pd
import glob
!pip install scikit-image
from skimage.registration import phase_cross_correlation



Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-8).
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 32.7 MB/s eta 0:00:00


In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

# Define paths
uploaded_file_path = '/content/drive/MyDrive/schizconnect_COBRE_images_22498.7z.001'
output_dir = '/content/extracted_data'
os.makedirs(output_dir, exist_ok=True)

print("Google Drive mounted and paths defined.")


Mounted at /content/drive
Google Drive mounted and paths defined.


In [ ]:
# Rename the file to .7z
renamed_file_path = uploaded_file_path.replace('.7z.001', '.7z')
os.rename(uploaded_file_path, renamed_file_path)

print(f"File renamed to: {renamed_file_path}")


File renamed to: /content/drive/MyDrive/schizconnect_COBRE_images_22498.7z


In [ ]:
# Extract the renamed .7z file
!7z x "/content/drive/MyDrive/schizconnect_COBRE_images_22498.7z" -o"/content/extracted_data"


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/drive/MyDrive/                                 1 file, 2277077170 bytes (2172 MiB)

Extracting archive: /content/drive/MyDrive/schizconnect_COBRE_images_22498.7z
--
Path = /content/drive/MyDrive/schizconnect_COBRE_images_22498.7z
Type = 7z
Physical Size = 2277077170
Headers Size = 13601
Method = LZMA2:24
Solid = +
Blocks = 2

  0%      1% 318 - COBRE/sub-A00000456/ses-20090 . 20090101_task-rest_bold.nii.gz                                                                           2% 328 - COBRE/sub-A00000541/ses-20100 . 20100

In [ ]:
# List the extracted files
'''for root, dirs, files in os.walk(output_dir):
    for file in files:
        print(os.path.join(root, file))'''

save_dir = '/content/fmri/corrected_nifti_data/'
os.makedirs(save_dir, exist_ok=True)

process_dir = '/content/fmri/processed_imaging_data/'
os.makedirs(process_dir, exist_ok=True)

spatial_maps_dir = '/content/fmri/spatial_maps_data/'
os.makedirs(spatial_maps_dir, exist_ok=True)
ica = CanICA(n_components=20, mask_strategy='background')
scaler = StandardScaler()

In [ ]:
import os
import glob
import nibabel as nib
import numpy as np
from nilearn.image import resample_img, resample_to_img, smooth_img, clean_img
from nilearn.masking import compute_brain_mask
from nilearn.input_data import NiftiMasker
from nilearn.datasets import load_mni152_template
from skimage.registration import phase_cross_correlation
from scipy.ndimage import shift
from sklearn.preprocessing import StandardScaler
from joblib import Parallel, delayed
from nilearn.maskers import NiftiMasker
from nilearn.decomposition import CanICA

# Initialize scaler globally for reuse
scaler = StandardScaler()

import os
import glob
import nibabel as nib
import numpy as np
from nilearn.image import resample_to_img, smooth_img, clean_img
from nilearn.datasets import load_mni152_template, fetch_atlas_harvard_oxford
from nilearn.maskers import NiftiLabelsMasker
from nilearn.input_data import NiftiMasker
from sklearn.preprocessing import StandardScaler

# Load Harvard-Oxford Atlas (ROIs)
ho_atlas = fetch_atlas_harvard_oxford('cort-maxprob-thr25-2mm')  # 96 cortical ROIs
ho_masker = NiftiLabelsMasker(labels_img=ho_atlas.maps, standardize=True, detrend=True,
                             low_pass=0.1, high_pass=0.01, t_r=2.0)

def extract_roi_features(nifti_img):
    """Extract mean ROI time-series features from fMRI data."""
    # Motion correction (use Nilearn's default instead of custom)
    # Note: If motion correction is already done, skip this step.
    cleaned_img = clean_img(nifti_img, detrend=True, standardize="zscore", t_r=2.0)

    # Smoothing
    smoothed_img = smooth_img(cleaned_img, fwhm=6)

    # Extract ROI time-series (shape: [time_points, n_rois])
    roi_time_series = ho_masker.fit_transform(smoothed_img)

    # Feature engineering: Temporal variability (mean/std across time)
    roi_means = np.mean(roi_time_series, axis=0)   # Mean activation per ROI
    roi_stds = np.std(roi_time_series, axis=0)     # Temporal variability per ROI

    # Combine into feature vector (shape: [n_samples, n_features])
    features = np.concatenate([roi_means, roi_stds])  # 96*2 = 192 features

    return features

# Main processing loop (simplified)
def process_nifti_files(nifti_files):
    features = []
    for i, nii_file in enumerate(nifti_files):
        print(f"Processing {i+1}/{len(nifti_files)}: {nii_file}")
        img = nib.load(nii_file)
        features.append(extract_roi_features(img))
    return np.array(features)

nifti_file_path = '/content/extracted_data/COBRE/sub*/ses*/func/*.nii.gz'

# Glob all files
nifti_files = glob.glob(nifti_file_path)
# Debugging step to print the files being processed
def is_empty_nifti(file_path):
    img = nib.load(file_path)
    return np.count_nonzero(img.get_fdata()) == 0  # Check if data is empty

nifti_files = glob.glob(nifti_file_path)
nifti_files = [f for f in nifti_files if not is_empty_nifti(f)]
def is_valid_nifti(file_path):
    try:
        nib.load(file_path)
        return True
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return False

nifti_files = glob.glob(nifti_file_path)
nifti_files = [f for f in nifti_files if is_valid_nifti(f)]

print("NIfTI files found:", nifti_files)

# Run the pipeline
fmri_features = process_nifti_files(nifti_files, save_dir, process_dir)

print("Preprocessing complete!")


NIfTI files found: ['/content/extracted_data/COBRE/sub-A00014590/ses-20110101/func/sub-A00014590_ses-20110101_task-rest_bold.nii.gz', '/content/extracted_data/COBRE/sub-A00037224/ses-20130101/func/sub-A00037224_ses-20130101_task-rest_bold.nii.gz', '/content/extracted_data/COBRE/sub-A00029486/ses-20120101/func/sub-A00029486_ses-20120101_task-rest_bold.nii.gz', '/content/extracted_data/COBRE/sub-A00015518/ses-20100101/func/sub-A00015518_ses-20100101_task-rest_bold.nii.gz', '/content/extracted_data/COBRE/sub-A00016197/ses-20090101/func/sub-A00016197_ses-20090101_task-rest_bold.nii.gz', '/content/extracted_data/COBRE/sub-A00027119/ses-20100101/func/sub-A00027119_ses-20100101_task-rest_bold.nii.gz', '/content/extracted_data/COBRE/sub-A00000838/ses-20100101/func/sub-A00000838_ses-20100101_task-rest_bold.nii.gz', '/content/extracted_data/COBRE/sub-A00000541/ses-20110101/func/sub-A00000541_ses-20110101_task-rest_bold.nii.gz', '/content/extracted_data/COBRE/sub-A00000541/ses-20100101/func/sub-A

/usr/local/lib/python3.11/dist-packages/nilearn/decomposition/_base.py:492: UserWarning: memory_level is currently set to 0 but a Memory object has been provided. Setting memory_level to 1.
  self.masker_.fit()
/usr/local/lib/python3.11/dist-packages/nilearn/decomposition/canica.py:292: UserWarning: memory_level is currently set to 0 but a Memory object has been provided. Setting memory_level to 1.
  components = _MultiPCA._raw_fit(self, data)
<ipython-input-6-0752c35b9a67>:115: FutureWarning: The nifti_maps_masker_ attribute is deprecated andwill be removed in Nilearn 0.11.3. Please use maps_masker_ instead.
  ica.fit(smoothed_img)


Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 490)
Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 443)
Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 468)
Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 472)
Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 513)
Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 448)
Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 517)
Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 498)
Checking masker...
Checking smoothed image...
Applying masker transformation...


/usr/local/lib/python3.11/dist-packages/nilearn/decomposition/_base.py:492: UserWarning: memory_level is currently set to 0 but a Memory object has been provided. Setting memory_level to 1.
  self.masker_.fit()
/usr/local/lib/python3.11/dist-packages/nilearn/decomposition/canica.py:292: UserWarning: memory_level is currently set to 0 but a Memory object has been provided. Setting memory_level to 1.
  components = _MultiPCA._raw_fit(self, data)
<ipython-input-6-0752c35b9a67>:115: FutureWarning: The nifti_maps_masker_ attribute is deprecated andwill be removed in Nilearn 0.11.3. Please use maps_masker_ instead.
  ica.fit(smoothed_img)


Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 491)
Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 793)
Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 418)
Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 532)
Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 327)
Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 428)
Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 456)
Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 428)
Checking masker...
Checking smoothed image...
Applying masker transformation...


...

Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 555)
Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 467)
Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 486)
Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 490)
Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 467)
Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 537)
Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 510)
Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 501)
Checking masker...
Checking smoothed image...
Applying masker transformation...


/usr/local/lib/python3.11/dist-packages/nilearn/decomposition/_base.py:492: UserWarning: memory_level is currently set to 0 but a Memory object has been provided. Setting memory_level to 1.
  self.masker_.fit()
/usr/local/lib/python3.11/dist-packages/nilearn/decomposition/canica.py:292: UserWarning: memory_level is currently set to 0 but a Memory object has been provided. Setting memory_level to 1.
  components = _MultiPCA._raw_fit(self, data)
<ipython-input-6-0752c35b9a67>:115: FutureWarning: The nifti_maps_masker_ attribute is deprecated andwill be removed in Nilearn 0.11.3. Please use maps_masker_ instead.
  ica.fit(smoothed_img)


Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 401)
Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 398)
Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 149)
Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 383)
Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 364)
Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 411)
Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 371)
Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 368)
Checking masker...
Checking smoothed image...
Applying masker transformation...


/usr/local/lib/python3.11/dist-packages/nilearn/decomposition/_base.py:492: UserWarning: memory_level is currently set to 0 but a Memory object has been provided. Setting memory_level to 1.
  self.masker_.fit()
/usr/local/lib/python3.11/dist-packages/nilearn/decomposition/canica.py:292: UserWarning: memory_level is currently set to 0 but a Memory object has been provided. Setting memory_level to 1.
  components = _MultiPCA._raw_fit(self, data)
<ipython-input-6-0752c35b9a67>:115: FutureWarning: The nifti_maps_masker_ attribute is deprecated andwill be removed in Nilearn 0.11.3. Please use maps_masker_ instead.
  ica.fit(smoothed_img)


Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 507)
Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 522)
Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 517)
Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 556)
Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 497)
Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 468)
Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 481)
Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 529)
Checking masker...
Checking smoothed image...
Applying masker transformation...


In [ ]:
print(fmri_features.shape)

In [ ]:
import zipfile
# Mount Google Drive
drive.mount('/content/drive')
# Define paths
uploaded_file_path = '/content/drive/MyDrive/dataverse_files.zip'
data_dir = '/content/extracted_eeg_data'
os.makedirs(data_dir, exist_ok=True)


# Extract the contents of the zip file
with zipfile.ZipFile(uploaded_file_path, 'r') as zip_ref:
    zip_ref.extractall(data_dir)
print("Google Drive mounted and paths defined.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive mounted and paths defined.


In [ ]:
import os
import numpy as np
import mne
from mne.io import read_raw_edf
from mne.channels import make_standard_montage

# Constants
DATA_DIR = "/path/to/edf/files"  # Update this
OUTPUT_DIR = "/path/to/eeg_features"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Define electrode groups for brain regions (simplified)
REGIONS = {
    "frontal": ["Fp1", "Fp2", "F3", "F4", "F7", "F8"],
    "central": ["C3", "C4", "Cz"],
    "temporal": ["T3", "T4", "T5", "T6"],
    "parietal": ["P3", "P4", "Pz"],
    "occipital": ["O1", "O2"]
}

BANDS = {
    "delta": (0.5, 4),
    "theta": (4, 8),
    "alpha": (8, 12),
    "beta": (12, 30)
}

def extract_eeg_features(raw, label):
    """Extract region-specific spectral power (mean + std) per subject."""
    # Preprocess
    raw.pick_types(eeg=True)
    raw.set_montage(make_standard_montage("standard_1020"))  # Ensure electrode positions
    raw.filter(0.5, 30., fir_design='firwin')  # Skip gamma (noisy in EEG)

    # Extract epochs (non-overlapping 2s windows)
    epochs = mne.make_fixed_length_epochs(raw, duration=2, preload=True)

    # Compute PSD for all epochs
    psds, freqs = mne.time_frequency.psd_array_welch(
        epochs.get_data(),
        sfreq=raw.info["sfreq"],
        fmin=0.5,
        fmax=30,
        n_fft=256
    )

    # Aggregate features per region and band
    features = []
    for region, channels in REGIONS.items():
        # Find indices of channels in the region
        chan_indices = [i for i, ch in enumerate(raw.ch_names) if ch in channels]
        if not chan_indices:
            continue  # Skip if no channels found

        for band, (fmin, fmax) in BANDS.items():
            # Compute mean power across epochs and channels in the region
            band_mask = (freqs >= fmin) & (freqs < fmax)
            band_power = psds[:, chan_indices, :][..., band_mask].mean(axis=(0, 1, 3))
            band_std = psds[:, chan_indices, :][..., band_mask].std(axis=(0, 1, 3))

            features.extend([band_power, band_std])

    # Add label and save (one file per subject)
    features.append(label)
    output_file = os.path.join(OUTPUT_DIR, f"{os.path.splitext(file_name)[0]}_features.npy")
    np.save(output_file, np.array(features))

# Process files
for file_name in os.listdir(DATA_DIR):
    if file_name.endswith(".edf"):
        file_path = os.path.join(DATA_DIR, file_name)
        try:
            raw = read_raw_edf(file_path, preload=True)
            label = 0 if file_name.startswith("h") else 1  # Update label logic
            extract_eeg_features(raw, label)
        except Exception as e:
            print(f"Error processing {file_name}: {e}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 67.0 MB/s eta 0:00:00
Processing file: /content/extracted_eeg_data/h10.edf
Extracting EDF parameters from /content/extracted_eeg_data/h10.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 278749  =      0.000 ...  1114.996 secs...
Assigned label 0 for file h10.edf
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 1651 samp

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s


Shape of features: (557, 95), Label: 0, Final shape: (557, 96)
Saved features to /content/eeg_features/h10_features.npy
Processing file: /content/extracted_eeg_data/h03.edf
Extracting EDF parameters from /content/extracted_eeg_data/h03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 227499  =      0.000 ...   909.996 secs...
Assigned label 0 for file h03.edf
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff freq

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


Shape of features: (454, 95), Label: 0, Final shape: (454, 96)
Saved features to /content/eeg_features/h03_features.npy
Processing file: /content/extracted_eeg_data/h12.edf
Extracting EDF parameters from /content/extracted_eeg_data/h12.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 224999  =      0.000 ...   899.996 secs...
Assigned label 0 for file h12.edf
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff freq

...

Shape of features: (455, 95), Label: 1, Final shape: (455, 96)
Saved features to /content/eeg_features/s08_features.npy
Processing file: /content/extracted_eeg_data/h07.edf
Extracting EDF parameters from /content/extracted_eeg_data/h07.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 227499  =      0.000 ...   909.996 secs...
Assigned label 0 for file h07.edf
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff freq

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


Shape of features: (454, 95), Label: 0, Final shape: (454, 96)
Saved features to /content/eeg_features/h07_features.npy
Processing file: /content/extracted_eeg_data/h09.edf
Extracting EDF parameters from /content/extracted_eeg_data/h09.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 226249  =      0.000 ...   904.996 secs...
Assigned label 0 for file h09.edf
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff freq

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s


Shape of features: (452, 95), Label: 0, Final shape: (452, 96)
Saved features to /content/eeg_features/h09_features.npy


In [ ]:
import os
import numpy as np

def load_eeg_features(data_dir):
    """
    Load EEG features aggregated per subject
    Returns:
        eeg_features: np.array of shape [n_subjects, n_features]
        eeg_labels: np.array of shape [n_subjects]
    """
    subject_features = []
    subject_labels = []

    # Validate directory
    if not os.path.isdir(data_dir):
        raise ValueError(f"Invalid data directory: {data_dir}")

    # Load and process each file
    for file_name in sorted(os.listdir(data_dir)):
        if file_name.endswith("_features.npy"):
            try:
                file_path = os.path.join(data_dir, file_name)

                # Load data
                data = np.load(file_path)

                # Validate data shape
                if data.ndim != 2 or data.shape[1] < 2:
                    raise ValueError(f"Invalid shape {data.shape} in {file_name}")

                # Separate features and labels
                features = data[:, :-1]  # All columns except last
                labels = data[:, -1]     # Last column as labels

                # Verify consistent labels per subject
                unique_labels = np.unique(labels)
                if len(unique_labels) != 1:
                    raise ValueError(f"Multiple labels found in {file_name}")

                # Average across epochs (assuming rows=epochs)
                subject_features.append(np.mean(features, axis=0))
                subject_labels.append(unique_labels[0])

            except Exception as e:
                print(f"Skipping {file_name} due to error: {str(e)}")
                continue

    # Convert to numpy arrays
    eeg_features = np.array(subject_features)
    eeg_labels = np.array(subject_labels)

    # Final validation
    if len(eeg_features) == 0:
        raise ValueError("No valid EEG data found")

    if eeg_features.shape[0] != eeg_labels.shape[0]:
        raise ValueError("Feature/label count mismatch")

    print(f"Loaded {len(eeg_features)} subjects with {eeg_features.shape[1]} features")
    return eeg_features, eeg_labels

# Usage example
output_dir = "/content/eeg_features/"
eeg_features, eeg_labels = load_eeg_features(output_dir)

print("EEG features shape:", eeg_features.shape)  # [n_subjects, n_features]
print("Labels shape:", eeg_labels.shape)         # [n_subjects]

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import StratifiedKFold, train_test_split

# =====================
# 1. Data Preparation
# =====================

class DomainAdaptationDataset(Dataset):
    def __init__(self, source_features, source_labels, target_features):
        self.source_features = source_features
        self.source_labels = source_labels
        self.target_features = target_features

    def __len__(self):
        return max(len(self.source_features), len(self.target_features))

    def __getitem__(self, idx):
        source_idx = idx % len(self.source_features)
        target_idx = idx % len(self.target_features)

        return {
            'source': (self.source_features[source_idx], self.source_labels[source_idx]),
            'target': (self.target_features[target_idx], -1)  # Dummy label
        }

# =====================
# 2. DANN Architecture
# =====================

class GradientReversalLayer(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha
        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        return grad_output.neg() * ctx.alpha, None

class DANN(nn.Module):
    def __init__(self, input_dim, hidden_dim=64):
        super(DANN, self).__init__()
        self.feature_extractor = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU()
        )
        self.task_classifier = nn.Sequential(
            nn.Linear(hidden_dim, 1),
            nn.Sigmoid()
        )
        self.domain_classifier = nn.Sequential(
            nn.Linear(hidden_dim, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, x, alpha=1.0):
        features = self.feature_extractor(x)
        task_output = self.task_classifier(features)
        reversed_features = GradientReversalLayer.apply(features, alpha)
        domain_output = self.domain_classifier(reversed_features)
        return task_output, domain_output

# =====================
# 3. Training Loop with Validation
# =====================

def train_dann(eeg_train, labels_train, fmri_features, eeg_val, labels_val, num_epochs=100, lambda_val=0.1):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Standardize features
    scaler = StandardScaler()
    combined = np.vstack([eeg_train, fmri_features])
    scaler.fit(combined)

    eeg_train_scaled = scaler.transform(eeg_train)
    eeg_val_scaled = scaler.transform(eeg_val)
    fmri_scaled = scaler.transform(fmri_features)

    # Create dataset and dataloader
    dataset = DomainAdaptationDataset(
        source_features=eeg_train_scaled,
        source_labels=labels_train,
        target_features=fmri_scaled
    )
    dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

    # Initialize model
    model = DANN(input_dim=eeg_train_scaled.shape[1]).to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    task_loss_fn = nn.BCELoss()
    domain_loss_fn = nn.BCELoss()

    best_val_acc = 0.0
    best_model_state = None

    for epoch in range(num_epochs):
        model.train()
        total_task_loss = 0
        total_domain_loss = 0

        for batch in dataloader:
            # Training loop
            source_data, source_labels = batch['source']
            target_data, _ = batch['target']

            source_data = source_data.float().to(device)
            source_labels = source_labels.float().to(device)
            target_data = target_data.float().to(device)

            combined_data = torch.cat([source_data, target_data], dim=0)
            batch_size = source_data.size(0)

            domain_labels = torch.cat([
                torch.zeros(batch_size),
                torch.ones(combined_data.size(0) - batch_size)
            ]).to(device)

            task_preds, domain_preds = model(combined_data)
            source_task_preds = task_preds[:batch_size]

            task_loss = task_loss_fn(source_task_preds.squeeze(), source_labels)
            domain_loss = domain_loss_fn(domain_preds.squeeze(), domain_labels)
            total_loss = task_loss + lambda_val * domain_loss

            optimizer.zero_grad()
            total_loss.backward()
            optimizer.step()

            total_task_loss += task_loss.item()
            total_domain_loss += domain_loss.item()

        # Validation
        model.eval()
        with torch.no_grad():
            val_data = torch.tensor(eeg_val_scaled).float().to(device)
            val_preds, _ = model(val_data)
            val_preds = val_preds.squeeze().cpu().numpy().round()
            val_acc = (val_preds == labels_val).mean()

            if val_acc > best_val_acc:
                best_val_acc = val_acc
                best_model_state = model.state_dict()

        print(f'Epoch {epoch+1}: Val Acc {val_acc:.2f}')

    # Load best model
    model.load_state_dict(best_model_state)
    return model, scaler

# =====================
# 4. Evaluation Functions
# =====================

def evaluate_metrics(model, scaler, features, labels, name):
    model.eval()
    with torch.no_grad():
        scaled_features = scaler.transform(features)
        features_tensor = torch.tensor(scaled_features).float().to(device)
        predictions, _ = model(features_tensor)
        preds = predictions.squeeze().cpu().numpy().round()

    acc = (preds == labels).mean()
    cm = confusion_matrix(labels, preds)
    report = classification_report(labels, preds, zero_division=0)

    print(f"\n{name} Evaluation:")
    print(f"Accuracy: {acc*100:.2f}%")
    print("Classification Report:")
    print(report)
    print("Confusion Matrix:")
    print(cm)

    return acc, cm, report

# =====================
# 5. Cross-Validation Pipeline
# =====================

if __name__ == "__main__":
    # Load data
    eeg_features = np.load('/path/to/eeg_features.npy')
    eeg_labels = np.load('/path/to/eeg_labels.npy')
    fmri_features = np.load('/path/to/fmri_features.npy')
    fmri_labels = np.ones(len(fmri_features))

    # Initialize CV
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    results = []

    for fold_idx, (train_val_idx, test_idx) in enumerate(kf.split(eeg_features, eeg_labels)):
        print(f"\n=== Fold {fold_idx+1} ===")

        # Split into train/val/test
        eeg_train_val = eeg_features[train_val_idx]
        labels_train_val = eeg_labels[train_val_idx]
        eeg_test = eeg_features[test_idx]
        labels_test = eeg_labels[test_idx]

        # Further split train_val into train/val
        eeg_train, eeg_val, labels_train, labels_val = train_test_split(
            eeg_train_val, labels_train_val,
            test_size=0.25,
            stratify=labels_train_val,
            random_state=42
        )

        # Train model
        model, scaler = train_dann(
            eeg_train, labels_train,
            fmri_features,
            eeg_val, labels_val,
            num_epochs=100,
            lambda_val=0.1
        )

        # Evaluate
        test_acc, test_cm, test_report = evaluate_metrics(model, scaler, eeg_test, labels_test, "EEG Test")
        fmri_acc, fmri_cm, fmri_report = evaluate_metrics(model, scaler, fmri_features, fmri_labels, "fMRI")

        results.append({
            'fold': fold_idx+1,
            'test_acc': test_acc,
            'fmri_acc': fmri_acc,
            'test_cm': test_cm,
            'fmri_cm': fmri_cm
        })

    # Final report
    print("\n=== Final Results ===")
    print(f"Mean EEG Test Accuracy: {np.mean([r['test_acc'] for r in results])*100:.2f}%")
    print(f"Mean fMRI Accuracy: {np.mean([r['fmri_acc'] for r in results])*100:.2f}%")
    print("\nAggregated EEG Test Confusion Matrix:")
    print(sum([r['test_cm'] for r in results]))
    print("\nAggregated fMRI Confusion Matrix:")
    print(sum([r['fmri_cm'] for r in results]))

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight

# -------------------
# Data Preprocessing
# -------------------
def pad_or_truncate(data, target_size):
    """Pads with zeros or truncates the dataset to match the required size."""
    current_size = data.shape[0]
    if current_size < target_size:
        pad_size = target_size - current_size
        pad_shape = (pad_size,) + data.shape[1:]  # Keep feature dimensions unchanged
        pad_values = np.zeros(pad_shape)  # Pad with zeros
        return np.vstack((data, pad_values))  # Stack padded data
    return data[:target_size]  # Truncate if larger

# Reshape fMRI features
fmri_features = fmri_features_old.reshape(fmri_features_old.shape[0], -1)

# Align EEG and fMRI sample sizes
num_samples = max(eeg_features.shape[0], fmri_features.shape[0])
#X_eeg = pad_or_truncate(eeg_features, num_samples)
X_fmri = pad_or_truncate(fmri_features, num_samples)
y_target = pad_or_truncate(y.reshape(-1, 1), num_samples).flatten()

pca = PCA(n_components=eeg_features.shape[1])
fmri_aligned = pca.fit_transform(X_fmri)

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight

# -------------------
# Data Preprocessing
# -------------------
def pad_or_truncate(data, target_size):
    """Pads with zeros or truncates the dataset to match the required size."""
    current_size = data.shape[0]
    if current_size < target_size:
        pad_size = target_size - current_size
        pad_shape = (pad_size,) + data.shape[1:]  # Keep feature dimensions unchanged
        pad_values = np.zeros(pad_shape)  # Pad with zeros
        return np.vstack((data, pad_values))  # Stack padded data
    return data[:target_size]  # Truncate if larger

# Reshape fMRI features
fmri_features = fmri_features_old.reshape(fmri_features_old.shape[0], -1)

# Align EEG and fMRI sample sizes
num_samples = max(eeg_features.shape[0], fmri_features.shape[0])
#X_eeg = pad_or_truncate(eeg_features, num_samples)
X_fmri = pad_or_truncate(fmri_features, num_samples)
y_target = pad_or_truncate(y.reshape(-1, 1), num_samples).flatten()  # Ensure labels are 1D

from sklearn.decomposition import PCA
# Combine EEG + fMRI

pca = PCA(n_components=eeg_features.shape[1])
fmri_aligned = pca.fit_transform(X_fmri)
X_combined = np.hstack((eeg_features, fmri_aligned))
# Standardize Data
scaler = StandardScaler()
X_combined = scaler.fit_transform(X_combined)

# -------------------
# Data Augmentation (Adding Noise)
# -------------------
def add_noise(data, noise_level=0.05):
    noise = np.random.normal(0, noise_level, data.shape)
    return data + noise

# Augment Data
X_noisy = add_noise(X_combined, noise_level=0.05)
X_combined_augmented = np.vstack((X_combined, X_noisy))
y_combined_augmented = np.hstack((y_target, y_target))

# Convert to PyTorch tensors
X_combined_augmented = torch.tensor(X_combined_augmented, dtype=torch.float32)
y_combined_augmented = torch.tensor(y_combined_augmented, dtype=torch.float32)  # Float for BCE loss

# Split into train/test
X_train, X_test, y_train, y_test = train_test_split(X_combined_augmented, y_combined_augmented, test_size=0.25, random_state=42)

# Convert to PyTorch tensors
X_train, X_test = torch.tensor(X_train, dtype=torch.float32), torch.tensor(X_test, dtype=torch.float32)
y_train, y_test = torch.tensor(y_train, dtype=torch.float32), torch.tensor(y_test, dtype=torch.float32)

# Create DataLoaders
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=64, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=64, shuffle=False)

# -------------------
# Define Fusion Model
# -------------------
class FusionModel(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(FusionModel, self).__init__()
        '''self.conv = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2),
            nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2),
        )

        conv_output_size = (input_dim // 4) * 128  # Adjust based on pooling layers
'''
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 1024),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(1024, 1)  # 1 output for binary classification
        )

    def forward(self, x):
        #x = x.unsqueeze(1)  # (batch_size, 1, feature_dim) for Conv1D
        #x = self.conv(x)
        #x = x.view(x.shape[0], -1)  # Flatten
        x = self.fc(x)
        return x  # No sigmoid here, we use BCEWithLogitsLoss


# Initialize the model
model = FusionModel(input_dim=X_combined_augmented.shape[1], hidden_dim=1024)

# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y_target), y=y_target)
class_weights = torch.tensor(class_weights, dtype=torch.float32)

# Define Loss Function (with class weights)
criterion = nn.BCEWithLogitsLoss(pos_weight=class_weights[1])

# Define Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Learning Rate Scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5, verbose=True)

# -------------------
# Training Loop
# -------------------
best_loss = float('inf')
for epoch in range(50):
    model.train()
    running_loss = 0.0

    for batch in train_loader:
        inputs, labels = batch
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs).squeeze(1)  # Ensure (batch_size,) shape
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}, Loss: {avg_loss:.4f}")

    # Save best model
    # Check for early stopping
    if avg_loss < best_loss:
        best_loss = avg_loss
        torch.save(model.state_dict(), 'best_model.pth')  # Save best model
    else:
        print("No improvement, applying LR scheduler")
        scheduler.step(avg_loss)  # Reduce learning rate if loss stops improving

# -------------------
# Evaluation
# -------------------
model.eval()
y_true, y_pred, y_probs = [], [], []

with torch.no_grad():
    for batch in test_loader:
        inputs, labels = batch
        outputs = model(inputs).squeeze(1)
        probs = torch.sigmoid(outputs)  # Convert logits to probabilities
        predictions = (probs > 0.5).float()  # Threshold at 0.5

        y_true.extend(labels.numpy())
        y_pred.extend(predictions.numpy())
        y_probs.extend(probs.numpy())

y_true, y_pred, y_probs = np.array(y_true), np.array(y_pred), np.array(y_probs)

# Metrics
# Compute evaluation metrics
accuracy = accuracy_score(y_true, y_pred)
classification_rep = classification_report(y_true, y_pred)
conf_matrix = confusion_matrix(y_true, y_pred)
roc_auc = roc_auc_score(y_true, y_probs)
TN, FP, FN, TP = confusion_matrix(y_true, y_pred).ravel()

print('True Positive(TP)  = ', TP)
print('False Positive(FP) = ', FP)
print('True Negative(TN)  = ', TN)
print('False Negative(FN) = ', FN)
# Print results
print(f"✅ Accuracy: {accuracy:.4f}")
print("\n🔹 Classification Report:\n", classification_rep)
print("\n🔹 Confusion Matrix:\n", conf_matrix)
print(f"\n🔹 ROC-AUC Score: {roc_auc:.4f}")
accuracy =  (TP + TN) / (TP + FP + TN + FN)
from sklearn.metrics import f1_score

f1 = f1_score(y_true, y_pred)
print(f"🔹 F1 Score: {f1:.4f}")
print('Accuracy of the binary classifier = {:0.3f}'.format(accuracy))


<ipython-input-29-ced868efb01a>:63: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train, X_test = torch.tensor(X_train, dtype=torch.float32), torch.tensor(X_test, dtype=torch.float32)
<ipython-input-29-ced868efb01a>:64: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train, y_test = torch.tensor(y_train, dtype=torch.float32), torch.tensor(y_test, dtype=torch.float32)
/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1, Loss: 0.6238
Epoch 2, Loss: 0.5751
Epoch 3, Loss: 0.5648
Epoch 4, Loss: 0.5539
Epoch 5, Loss: 0.5474
Epoch 6, Loss: 0.5403
Epoch 7, Loss: 0.5315
Epoch 8, Loss: 0.5292
Epoch 9, Loss: 0.5146
Epoch 10, Loss: 0.5045
Epoch 11, Loss: 0.4975
Epoch 12, Loss: 0.4806
Epoch 13, Loss: 0.4672
Epoch 14, Loss: 0.4736
No improvement, applying LR scheduler
Epoch 15, Loss: 0.4532
Epoch 16, Loss: 0.4400
Epoch 17, Loss: 0.4266
Epoch 18, Loss: 0.4116
Epoch 19, Loss: 0.4055
Epoch 20, Loss: 0.3976
Epoch 21, Loss: 0.3800
Epoch 22, Loss: 0.3688
Epoch 23, Loss: 0.3677
Epoch 24, Loss: 0.3573
Epoch 25, Loss: 0.3396
Epoch 26, Loss: 0.3286
Epoch 27, Loss: 0.3273
Epoch 28, Loss: 0.3411
No improvement, applying LR scheduler
Epoch 29, Loss: 0.3182
Epoch 30, Loss: 0.4557
No improvement, applying LR scheduler
Epoch 31, Loss: 0.3068
Epoch 32, Loss: 0.2986
Epoch 33, Loss: 0.2964
Epoch 34, Loss: 0.2871
Epoch 35, Loss: 0.2901
No improvement, applying LR scheduler
Epoch 36, Loss: 0.2891
No improvement, applying LR s

In [ ]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, f1_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample, compute_class_weight

# -------------------
# Data Preprocessing
# -------------------
'''print(eeg_features.shape)
print(fmri_features.shape)
print(y.shape)

# Ensure equal sample sizes
num_eeg_samples = eeg_features.shape[0]
num_fmri_samples = fmri_features.shape[0]
num_samples = max(num_eeg_samples, num_fmri_samples)

# Pad or truncate data
def pad_or_truncate(data, target_size):
    current_size = data.shape[0]
    if current_size < target_size:
        pad_size = target_size - current_size
        pad_shape = (pad_size,) + data.shape[1:]
        pad_values = np.zeros(pad_shape)
        return np.vstack((data, pad_values))
    return data[:target_size]

# Apply padding/truncation
X_eeg = pad_or_truncate(eeg_features, num_samples)
X_fmri = pad_or_truncate(fmri_features, num_samples)
y_target = pad_or_truncate(y.reshape(-1, 1), num_samples).flatten()

# Reshape fMRI features if necessary
X_fmri = X_fmri.reshape(num_samples, -1)

# Combine EEG and fMRI features
X_combined = np.hstack((X_eeg, X_fmri))

# Standardize data
ss = StandardScaler()
X_combined = ss.fit_transform(X_combined)'''

# -------------------
# Bootstrapping for Better Generalization
# -------------------
X_resampled, y_resampled = resample(X_combined, y_target, replace=True, n_samples=num_samples, random_state=42)

# -------------------
# K-Fold Cross-Validation (Leave 1 Fold for Final Testing)
# -------------------
from sklearn.model_selection import StratifiedKFold
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

splits = list(kf.split(X_resampled, y_resampled))  # Store indices to manually hold 1 fold for final testing

# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y), y=y)
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}

# Store metrics
all_accuracies, all_f1_scores, all_roc_auc = [], [], []

# Leave the last fold for final testing
train_folds = splits[:-1]  # First 4 folds for training
test_fold = splits[-1]     # Last fold for testing

# Cross-validation on first 4 folds
for fold, (train_idx, val_idx) in enumerate(train_folds):
    print(f"\nTraining fold {fold+1}")

    # Train-test split
    X_train, X_val = X_resampled[train_idx], X_resampled[val_idx]
    y_train, y_val = y_resampled[train_idx], y_resampled[val_idx]

    # Initialize Logistic Regression model
    model = LogisticRegression(C=0.01, class_weight='balanced', max_iter=500, solver='lbfgs')

    # Train the model
    model.fit(X_train, y_train)

    # Predictions
    y_pred = model.predict(X_val)
    y_probs = model.predict_proba(X_val)[:, 1]

    # Compute metrics
    accuracy = accuracy_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred)
    try:
        roc_auc = roc_auc_score(y_val, y_probs)
    except ValueError:
        roc_auc = None

    # Store results
    all_accuracies.append(accuracy)
    all_f1_scores.append(f1)
    if roc_auc is not None:
        all_roc_auc.append(roc_auc)

    # Print metrics
    print(f"🔹 Accuracy: {accuracy:.4f}")
    print(f"🔹 F1 Score: {f1:.4f}")
    if roc_auc is not None:
        print(f"🔹 ROC-AUC Score: {roc_auc:.4f}")
    print("🔹 Classification Report:\n", classification_report(y_val, y_pred))
    print("🔹 Confusion Matrix:\n", confusion_matrix(y_val, y_pred))

# -------------------
# Compute Average Metrics Across Training Folds
# -------------------
print(f"\n🔹 Average Accuracy: {np.mean(all_accuracies):.4f}")
print(f"🔹 Average F1 Score: {np.mean(all_f1_scores):.4f}")
if all_roc_auc:
    print(f"🔹 Average ROC-AUC: {np.mean(all_roc_auc):.4f}")

# -------------------
# Final Testing on Held-Out Fold
# -------------------
test_idx = test_fold[1]  # Get test set indices
X_test, y_test = X_resampled[test_idx], y_resampled[test_idx]

# Predict on test set
y_test_pred = model.predict(X_test)
y_test_probs = model.predict_proba(X_test)[:, 1]

# Compute final test metrics
test_accuracy = accuracy_score(y_test, y_test_pred)
test_f1 = f1_score(y_test, y_test_pred)
try:
    test_roc_auc = roc_auc_score(y_test, y_test_probs)
except ValueError:
    test_roc_auc = None

# Print final test metrics
print("\n🔹 Final Test Results on Held-Out Fold:")
print(f"🔹 Test Accuracy: {test_accuracy:.4f}")
print(f"🔹 Test F1 Score: {test_f1:.4f}")
if test_roc_auc is not None:
    print(f"🔹 Test ROC-AUC Score: {test_roc_auc:.4f}")
print("🔹 Final Test Classification Report:\n", classification_report(y_test, y_test_pred))
print("🔹 Final Test Confusion Matrix:\n", confusion_matrix(y_test, y_test_pred))



Training fold 1
🔹 Accuracy: 0.7183
🔹 F1 Score: 0.6879
🔹 ROC-AUC Score: 0.8146
🔹 Classification Report:
               precision    recall  f1-score   support

         0.0       0.63      0.90      0.74      1302
         1.0       0.88      0.57      0.69      1581

    accuracy                           0.72      2883
   macro avg       0.75      0.73      0.72      2883
weighted avg       0.77      0.72      0.71      2883

🔹 Confusion Matrix:
 [[1176  126]
 [ 686  895]]

Training fold 2
🔹 Accuracy: 0.7228
🔹 F1 Score: 0.6984
🔹 ROC-AUC Score: 0.8170
🔹 Classification Report:
               precision    recall  f1-score   support

         0.0       0.64      0.89      0.74      1301
         1.0       0.87      0.59      0.70      1581

    accuracy                           0.72      2882
   macro avg       0.75      0.74      0.72      2882
weighted avg       0.76      0.72      0.72      2882

🔹 Confusion Matrix:
 [[1158  143]
 [ 656  925]]

Training fold 3
🔹 Accuracy: 0.7144
🔹 F1

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.cluster import KMeans

# ... (Your previous code) ...

# Ensure all feature vectors are NumPy arrays and have the same length
#feature_vectors = [np.array(f) for f in feature_vectors]

# Find the maximum feature vector length
max_length = max(len(sublist) for f in feature_vectors for sublist in f)  # Modified to find max length across all sublists

# Pad all vectors to the same length
X = np.array([[np.pad(sublist, (0, max_length - len(sublist)), mode='constant')
              for sublist in f] for f in feature_vectors])  # Pad each sublist

print("Final feature matrix shape:", X.shape)  # Should be (num_samples, max_length)

# ... (Rest of your code) ...

Final feature matrix shape: (6, 50, 2196)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np
import os

# Load features from saved .npy files
def load_features(data_dir):
    feature_list = []
    labels_list = []

    # Ensure to use the correct directory variable (output_dir)
    for file_name in os.listdir(data_dir):
        if file_name.endswith("_features.npy"):  # Make sure we are looking at the correct files
            file_path = os.path.join(data_dir, file_name)
            try:
                features = np.load(file_path)
                feature_list.append(features[:, :-1])  # Exclude labels (everything except the last column)
                labels_list.append(features[:, -1])   # Labels are the last column (single column)

                print(f"Loaded {file_name}: features shape {features[:, :-1].shape}, labels shape {features[:, -1].shape}")

            except Exception as e:
                print(f"Error loading {file_name}: {e}")

    # Concatenate all features and labels from all files
    X = np.vstack(feature_list)  # Feature matrix (all rows, all feature columns)
    y = np.hstack(labels_list)   # Label vector (all labels)

    # Print the shapes of the final feature matrix and label vector
    print(f"Final shape of features matrix (X): {X.shape}")
    print(f"Final shape of labels vector (y): {y.shape}")

    return X, y


# Directory containing EEG feature files
output_dir = "/content/eeg_features/"
eeg_features, y = load_features(output_dir)

# Split into training and testing sets
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from torch.utils.data import DataLoader, TensorDataset

# Assuming X and y are already loaded correctly
# X: Features matrix (shape: 14411, 95)
# y: Labels vector (shape: 14411,)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(eeg_features, y, test_size=0.25, random_state=0)
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
X_train = ss.fit_transform(X_train)  # Fit & transform on train
X_test = ss.transform(X_test)

def add_noise(X, noise_level=0.01):
    noise = noise_level * np.random.randn(*X.shape)
    return X + noise

X_train_noisy = add_noise(X_train, noise_level=0.02)
X_train_combined = np.vstack((X_train, X_train_noisy))
y_train_combined = np.hstack((y_train, y_train))  # Duplicate labels

# Convert to tensors again
X_train_combined = torch.tensor(X_train_combined, dtype=torch.float32)
y_train_combined = torch.tensor(y_train_combined, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

# Create DataLoader for batching and shuffling
train_data = TensorDataset(X_train_combined, y_train_combined)
test_data = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

# Define the model
class EEGModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(EEGModel, self).__init__()
        self.eeg_branch = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.5),  # Dropout for regularization
            nn.Linear(256, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)  # Output layer for classification (2 classes)
        )

    def forward(self, x):
        return self.eeg_branch(x)

# Initialize the model
model = EEGModel(input_dim=eeg_features.shape[1], hidden_dim=256, output_dim=2)  # Adjust hidden_dim as needed

# Define loss function and optimizer
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.utils.class_weight import compute_class_weight

# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights = torch.tensor(class_weights, dtype=torch.float32)

# Update the loss function with class weights
criterion = nn.CrossEntropyLoss(weight=class_weights)

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop (use the same code you already have)
# Training loop (use the same code you already have)
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Add learning rate scheduler
scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5, verbose=True)

# Modify training loop
best_loss = float('inf')
for epoch in range(100):
    model.train()
    running_loss = 0.0

    for batch in train_loader:
        inputs, labels = batch
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}, Loss: {avg_loss}")

    # Check for early stopping
    if avg_loss < best_loss:
        best_loss = avg_loss
        torch.save(model.state_dict(), 'best_model.pth')  # Save best model
    else:
        print("No improvement, applying LR scheduler")
        scheduler.step(avg_loss)  # Reduce learning rate if loss stops improving


# Evaluate the model
import torch
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score

# Ensure model is in evaluation mode
model.eval()

# Store predictions and ground truth labels
y_true = []
y_pred = []
y_probs = []  # For probability scores

# Disable gradient computation for evaluation
with torch.no_grad():
    for batch in test_loader:
        inputs, labels = batch
        outputs = model(inputs)  # Get raw logits
        probs = torch.softmax(outputs, dim=1)[:, 1]  # Convert to probabilities (class 1)
        _, predicted = torch.max(outputs, 1)  # Get class with highest score

        y_true.extend(labels.numpy())  # Actual labels
        y_pred.extend(predicted.numpy())  # Predicted labels
        y_probs.extend(probs.numpy())  # Probability scores

# Convert lists to numpy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)
y_probs = np.array(y_probs)

# Compute evaluation metrics
accuracy = accuracy_score(y_true, y_pred)
classification_rep = classification_report(y_true, y_pred)
conf_matrix = confusion_matrix(y_true, y_pred)
roc_auc = roc_auc_score(y_true, y_probs)
TN, FP, FN, TP = confusion_matrix(y_true, y_pred).ravel()

print('True Positive(TP)  = ', TP)
print('False Positive(FP) = ', FP)
print('True Negative(TN)  = ', TN)
print('False Negative(FN) = ', FN)
# Print results
print(f"✅ Accuracy: {accuracy:.4f}")
print("\n🔹 Classification Report:\n", classification_rep)
print("\n🔹 Confusion Matrix:\n", conf_matrix)
print(f"\n🔹 ROC-AUC Score: {roc_auc:.4f}")
accuracy =  (TP + TN) / (TP + FP + TN + FN)
from sklearn.metrics import f1_score

f1 = f1_score(y_true, y_pred)
print(f"🔹 F1 Score: {f1:.4f}")
print('Accuracy of the binary classifier = {:0.3f}'.format(accuracy))




Loaded h07_features.npy: features shape (454, 95), labels shape (454,)
Loaded h12_features.npy: features shape (449, 95), labels shape (449,)
Loaded h04_features.npy: features shape (462, 95), labels shape (462,)
Loaded h08_features.npy: features shape (454, 95), labels shape (454,)
Loaded s05_features.npy: features shape (444, 95), labels shape (444,)
Loaded h14_features.npy: features shape (432, 95), labels shape (432,)
Loaded s01_features.npy: features shape (422, 95), labels shape (422,)
Loaded s07_features.npy: features shape (672, 95), labels shape (672,)
Loaded s02_features.npy: features shape (572, 95), labels shape (572,)
Loaded h13_features.npy: features shape (482, 95), labels shape (482,)
Loaded s04_features.npy: features shape (602, 95), labels shape (602,)
Loaded s14_features.npy: features shape (1084, 95), labels shape (1084,)
Loaded h03_features.npy: features shape (454, 95), labels shape (454,)
Loaded s06_features.npy: features shape (369, 95), labels shape (369,)
Load

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1, Loss: 0.6427615161478166
Epoch 2, Loss: 0.5587898084574198
Epoch 3, Loss: 0.5212043492046333
Epoch 4, Loss: 0.507072869930747
Epoch 5, Loss: 0.49530911357445123
Epoch 6, Loss: 0.4885310642641677
Epoch 7, Loss: 0.47668136003807454
Epoch 8, Loss: 0.4694664723421695
Epoch 9, Loss: 0.6478887118355057
No improvement, applying LR scheduler
Epoch 10, Loss: 0.45457844356813376
Epoch 11, Loss: 0.46505295569022026
No improvement, applying LR scheduler
Epoch 12, Loss: 0.44103509469850527
Epoch 13, Loss: 0.43192120116843274
Epoch 14, Loss: 0.4404962235653894
No improvement, applying LR scheduler
Epoch 15, Loss: 0.41914738886631453
Epoch 16, Loss: 0.4100108416475488
Epoch 17, Loss: 0.40240588546151945
Epoch 18, Loss: 0.39765056870745485
Epoch 19, Loss: 0.3914744392390082
Epoch 20, Loss: 0.38608605467532514
Epoch 21, Loss: 0.37885227534897936
Epoch 22, Loss: 0.4210988597933357
No improvement, applying LR scheduler
Epoch 23, Loss: 0.37167627249772733
Epoch 24, Loss: 0.37760312665672696
No im

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.cluster import KMeans


# Define Autoencoder class
class Autoencoder(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, latent_dim)
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.ReLU(),
            nn.Linear(128, input_dim)
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

# Initialize Autoencoder
print(X.shape, X.shape[0], X.shape[1])
input_dim = X.shape[2]  # Ensure correct shape after padding
latent_dim = 32
autoencoder = Autoencoder(input_dim, latent_dim)

# Set up optimizer and loss function
optimizer = optim.Adam(autoencoder.parameters(), lr=1e-3)
criterion = nn.MSELoss()

# Convert X to a PyTorch tensor
X_tensor = torch.FloatTensor(X)

# Train Autoencoder
num_epochs = 100
for epoch in range(num_epochs):
    autoencoder.train()
    optimizer.zero_grad()
    encoded, decoded = autoencoder(X_tensor)
    loss = criterion(decoded, X_tensor)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}")

# Extract latent features
autoencoder.eval()
with torch.no_grad():
    fmri_features_old, _ = autoencoder(X_tensor)

# Convert features to NumPy for clustering
fmri_features_np = fmri_features_old.numpy()
print('dim', fmri_features_np.shape)
# Apply K-Means Clustering
n_clusters = 2  # Choose the number of clusters
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
# Reshape to ensure it's (num_samples, latent_dim)
fmri_features_np = fmri_features_np.reshape(fmri_features_np.shape[0], -1)
cluster_labels = kmeans.fit_predict(fmri_features_np)

print("Cluster Labels:", cluster_labels)


(6, 50, 2196) 6 50
Epoch [10/100], Loss: 0.1345
Epoch [20/100], Loss: 0.0570
Epoch [30/100], Loss: 0.0385
Epoch [40/100], Loss: 0.0263
Epoch [50/100], Loss: 0.0201
Epoch [60/100], Loss: 0.0168
Epoch [70/100], Loss: 0.0147
Epoch [80/100], Loss: 0.0130
Epoch [90/100], Loss: 0.0116
Epoch [100/100], Loss: 0.0104
dim (6, 50, 32)
Cluster Labels: [0 0 0 0 1 0]
